### Bond valuations

The purpose of this page is to demonstrate bond valuation principles using Python. In this notebook we will show:

* Bond pricing using DCF
* Calculation of YTM

For the examples, we'll use a hypothetical 10 year Corporate bond which has a face of 100 and coupon 5. 

In [1]:
from __future__ import division

In [2]:
face = 100
coupon = 5
rate = 0.05
periods = 5

In [3]:
def pv(face, coupon, periods, rate, back_in = False):
    total_pv = 0
    discounted_cfs = []
    for n in range(1, periods+1):
        discounted_cfs.append((coupon / ((1 + rate)**n)))
    sum_coupons = sum(discounted_cfs)
    final_payment = (face / ((1 + rate)**n))    
    total_pv = sum_coupons + final_payment
    
    if back_in == True:
        print "Bond price breakdown \n-------------------"
        print "Discount rate: {}".format(rate)
        print "Periods: {}".format(periods)
        print "Coupon: {}\n".format(coupon)
        for period, i in enumerate(discounted_cfs, start=1):
            print "Discounted CF for Coupon in Year {} is {}".format(str(period), i)
        print "Discounted Principal is {}\n".format(final_payment)
        print "Bond price is: {}".format(total_pv)
        
    return total_pv

In [4]:
print pv(face, coupon, periods, rate, back_in=True)

Bond price breakdown 
-------------------
Discount rate: 0.05
Periods: 5
Coupon: 5

Discounted CF for Coupon in Year 1 is 4.7619047619
Discounted CF for Coupon in Year 2 is 4.53514739229
Discounted CF for Coupon in Year 3 is 4.31918799266
Discounted CF for Coupon in Year 4 is 4.11351237396
Discounted CF for Coupon in Year 5 is 3.91763083234
Discounted Principal is 78.3526166468

Bond price is: 100.0
100.0


In [5]:
print pv(face, coupon, periods, 0.08, back_in=False)

88.0218698888


In [6]:
price = pv(face, coupon, periods, rate)

def ytm(face, coupon, periods, price, back_in = False):
    ytm_val = coupon
    condition = True
    while condition:
        if price < face:
            ytm_val += 0.0001
        else:
            ytm_val -= 0.0001

        total_pv =  pv(face, coupon, periods, ytm_val)
    
        if total_pv > price:
            condition = True
        else:
            condition = False
    
        if back_in == True:
            print "A price of {} generates a YTM of {}".format(total_pv, ytm)
    
    return ytm_val
    

In [7]:
ytm(face, coupon, periods, price, back_in = False)

5.0001

In [8]:
def ytm_approx(face, coupon, price, periods):
    return ((coupon +((face - price)/ periods ))/ ((face + price) /2))

In [9]:
ytm_approx(face, coupon, price, periods)

0.050000000000000024

### Macaulay Duration

The Macaulay duration is calculed by multiplying the PV of each Cash Flow by the time it is recieved and diving by the price of the bond. 

In [10]:
face = 1000
coupon = 50
rate = 0.05
periods = 5

In [11]:
def mc_dur(face, coupon, periods, rate, back_in = False):
    
    price = pv(face, coupon, periods, rate)
    
    weighted_dcfs = []
    for n in range(1, periods+1):
        cf = ((coupon * n) / ((1 + rate)**n))
        weighted_dcfs.append(cf)
    
    weighted_principal = ((face * periods) / ((1 + rate)**periods))
    
    result = (sum(weighted_dcfs) + weighted_principal)/price
    
    if back_in == True:
        print 'Macaualy Duration Calculation\n------------------------------\n'
        for n, i in enumerate(weighted_dcfs, start = 1):
            print 'Weighted CF for Period {} is: {}\n'.format(n, round(i))
        print 'Weighted CF for Principal is: {}\n'.format(int(weighted_principal))
        print 'Total weighted CF is: {}'.format(int((sum(weighted_dcfs) + weighted_principal)))
        print 'Bond price is: {}'.format(price)
        print 'Macaualy Duration is {}'.format(mc_dur)
    
    return result
    

In [24]:
mc_dur(face, coupon, periods, rate, back_in=True)

Macaualy Duration Calculation
------------------------------

Weighted CF for Period 1 is: 48.0

Weighted CF for Period 2 is: 91.0

Weighted CF for Period 3 is: 130.0

Weighted CF for Period 4 is: 165.0

Weighted CF for Period 5 is: 196.0

Weighted CF for Principal is: 3917

Total weighted CF is: 4545
Bond price is: 1000.0
Macaualy Duration is <function mc_dur at 0x103ca8758>


4.54595050416236

In [31]:
# Modified duration = Macaulay Duration / (1 + YTM / number_of_coupons_per_year)

def modified_dur(coupons_per_year, face, coupon, periods, rate):
    
    mac = mc_dur(face, coupon, periods, rate)
    y = ytm(face, coupon, periods, price)
    
    mod_dur = mac / (1 + (y / coupons_per_year))
    
    return mod_dur

In [32]:
coupons_per_year = 1

In [33]:
modified_dur(coupons_per_year, face, coupon, periods, rate)

0.08913610961865487